In [1]:
import pandas as pd
import glob
import locale
import numpy as np
import numpy as np

locale.setlocale(locale.LC_ALL, 'fa_IR.UTF-8')


'fa_IR.UTF-8'

In [2]:
def quantile_score(vec, score):
    scorevec = np.zeros(len(vec))
    qu = np.quantile(vec, np.linspace(0, 1, score + 1))
    scorevec[(vec <= qu[1]) & (vec >= qu[0])] = 1
    for i in range(1, score - 1):
        scorevec[(vec <= qu[i + 1]) & (vec > qu[i])] = i + 1
    scorevec[vec > qu[score]] = score
    return scorevec



In [3]:
# Read Data
# Folder containing Parquet files
folder_path = "Data/data/*.parquet"

# List all parquet files
parquet_files = glob.glob(folder_path)

# Read and concatenate all files
df = pd.concat([pd.read_parquet(f) for f in parquet_files], ignore_index=True)


In [4]:
# Functions

def quantile_score(vec, score):
    scorevec = np.zeros(len(vec))
    qu = np.quantile(vec, np.linspace(0, 1, score + 1))
    scorevec[(vec <= qu[1]) & (vec >= qu[0])] = 1
    for i in range(1, score - 1):
        scorevec[(vec <= qu[i + 1]) & (vec > qu[i])] = i + 1
    scorevec[vec > qu[score]] = score
    return scorevec

def generate_dates(years, months, days30, days31):
    dates = []
    for month in months:
        if month in Months:
            dates.append(f"{years}{month}{days31}")
        else:
            dates.append(f"{years}{month}{days30}")
    return dates


In [5]:
# Parameters

start_date = 14030101  # Start train
target_date = 14030631  # End train

Years = ["1403"]
Months = ["{:02d}".format(i) for i in range(1, 13)]  # Generates "01" to "12"
Days30 = ["{:02d}".format(i) for i in range(1, 31)]  # Generates "01" to "30"
Days31 = ["{:02d}".format(i) for i in range(1, 32)]  # Generates "01" to "31"

print("Years:", Years)
print("Months:", Months)
print("Days30:", Days30)
print("Days31:", Days31)


Years: ['1403']
Months: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
Days30: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
Days31: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']


In [6]:
main_dates = generate_dates(Years, Months, Days30, Days31)


In [7]:
# Remove specific dates
excluded_dates = {14021031, 14021131, 14021231, 14030731, 14030831, 14030931, 14031031, 14031131, 14031231}
main_dates = [date for date in main_dates if date not in excluded_dates]

In [8]:
DimDate = pd.read_csv("Data/data/DimDate.csv")

In [10]:
import pandas as pd

# Convert date column to string
df["date_CHR"] = df["date"].astype(str)

# Create Miladi_Num by extracting and concatenating substrings
df["Miladi_Num"] = df["date_CHR"].str[:4] + df["date_CHR"].str[5:7] + df["date_CHR"].str[8:10]

# Convert Miladi column to string
DimDate["Miladi_CHR"] = DimDate["Miladi"].astype(str)

# Create Miladi_Num in DimDate
DimDate["Miladi_Num"] = DimDate["Miladi_CHR"].str[:4] + DimDate["Miladi_CHR"].str[5:7] + DimDate["Miladi_CHR"].str[8:10]

# Select specific columns
DimDateS = DimDate[["Jalali_1", "Miladi_Num"]]

# Left join on Miladi_Num
df = df.merge(DimDateS, on="Miladi_Num", how="left")

# Create Shamsi_Date and convert to numeric
df["Shamsi_Date_Num"] = (df["Jalali_1"].str[:4] + df["Jalali_1"].str[5:7] + df["Jalali_1"].str[8:10]).astype(int)


In [11]:
# Assuming df is already a pandas DataFrame
TrainRFM = df[df["module"] == "OnlineShopping"]

TrainRFM = TrainRFM[TrainRFM["payment_status"] == "payed"]

In [12]:
TrainRFM.columns

Index(['_id', 'user_id', 'date', 'client_type', 'status', 'payment_status',
       'type', 'module', 'product_name', 'initial_total', 'product_price',
       'converted_value', 'qty', 'module_unit_price', 'Amount_in_dollars',
       'first_purchased', 'date_CHR', 'Miladi_Num', 'Jalali_1',
       'Shamsi_Date_Num'],
      dtype='object')